In [1]:
#!/usr/bin/env python3
import pympi as pmp
import os
import json

TAG_SPANISH = "[- spa]"
TAG_UNDEFINED = "[- und]"
TAG_MIX = "[- mix]"
TAG_CHEQUEAR = "chequear"

class EafFile:
    def _getParticipants(self):
        self.participants = []

        for k in self.eafFile.tiers.keys():
            if 'PARTICIPANT' in self.eafFile.tiers[k][2].keys():
                if self.eafFile.tiers[k][2]['PARTICIPANT'] not in self.participants:
                    self.participants.append(self.eafFile.tiers[k][2]['PARTICIPANT'])

        # print('participants: {}'.format(self.participants))
    def _getTiers(self):
        self.tiers = []

        for k in self.eafFile.tiers.keys():
            if 'TIER_ID' in self.eafFile.tiers[k][2].keys():
                if self.eafFile.tiers[k][2]['TIER_ID'] not in self.tiers:
                    self.tiers.append(self.eafFile.tiers[k][2]['TIER_ID'])

        # print('tiers: {}'.format(self.tiers))

    def __init__(self, eafFilePath):
        self.eafPath = eafFilePath
        self.eafFile = pmp.Elan.Eaf(eafFilePath)

        self._getParticipants()
        self._getTiers()

    def segmentAudioByKeyword(self, audio_file_path, keyword, outputPath):
        from pydub import AudioSegment

        audio = AudioSegment.from_file(audio_file_path)
        segmentos = []

        for participant in self.tiers:
            for segmento, val in self.eafFile.tiers[participant][0].items():
                start = val[0]
                end = val[1]
                uttr = val[2].strip()

                if uttr == keyword:
                    t0 = self.eafFile.timeslots[start]
                    t1 = self.eafFile.timeslots[end]
                    # length = (t1 - t0) / sampling_freq

                    filename = f"{participant}_{t0}_{t1}.wav"
                    
                    segmentos.append({ "segmento" : segmento, "filename" : filename })

                    audioSlice = audio[t0:t1]
                    audioSlice.export(f"{outputPath}/{filename}", format="wav")
        
        return segmentos

    #by language
    def segmentAudio(self, audio_file_path, skipTiers = ["dudas", "code", "code_num","on_off","note","candidato", "code-1"]):
        from pydub import AudioSegment

        audio = AudioSegment.from_file(audio_file_path)
        sampling_freq = 1000

        base = os.path.basename(self.eafPath)
        dirname = os.path.dirname(self.eafPath)
        name = os.path.splitext(base)[0]

        print('parsing file: {}'.format(name))

        count = {
            "spa" : { "utterances" : 0, "seconds" : 0 },
            "qom" : { "utterances" : 0, "seconds" : 0 },
            "und" : { "utterances" : 0, "seconds" : 0 },
#            "unk" : { "utterances" : 0, "seconds" : 0 }
        }

        for d in count:
            os.makedirs(f"{dirname}/{name}_slices/{d}", exist_ok=True)

        with open(os.path.join(dirname, name + ".stats.json"), "w") as stats:
#            tsv.write("time\tlength\tlanguage")

            # for participant in self.participants:
            for participant in self.tiers:
                skip = False
                for t in skipTiers:
                    if t.lower() in participant.lower():
                        print(f"Skipping tier '{participant}'")
                        skip = True
                        break
                if skip: continue

                print(f"Processing tier '{participant}'")
                for _, val in self.eafFile.tiers[participant][0].items():
                    start = val[0]
                    end = val[1]
                    uttr = val[2].strip()

                    t0 = self.eafFile.timeslots[start]
                    t1 = self.eafFile.timeslots[end]
                    length = (t1 - t0) / sampling_freq

                    uttrLanguage = "qom"
                    # print(f"'{uttr}'")

                    # if uttr == "":
                    #    uttrLanguage = "qom"
                    # el
                    if TAG_SPANISH in uttr:
                        uttrLanguage = "spa"
                    elif TAG_UNDEFINED in uttr :
                        uttrLanguage = "und"
                    elif TAG_CHEQUEAR in uttr or TAG_MIX in uttr:
                        continue
                    # else:
                    #     print(f"Código inválido \"{uttr}\", descartado")
                    #     continue

                    count[uttrLanguage]["utterances"] += 1
                    count[uttrLanguage]["seconds"] += length

                    filename = f"{name}_slices/{uttrLanguage}/{participant}_{t0}_{t1}.wav"

                    audioSlice = audio[t0:t1]
                    audioSlice.export("%s/%s" % (dirname,filename), format="wav")


            json.dump(count, stats)
            print(f"{name} DONE :)")

    def toRTTM(self, path_to_write_rttm):
        """
        function to write a new .rttm file which is a transcription of the .eaf
        given as input
        """

        sampling_freq = 1000

        base = os.path.basename(self.eafPath)
        name = os.path.splitext(base)[0]

        print('parsing file: {}'.format(name))

        with open(os.path.join(path_to_write_rttm, name + ".rttm"), "w") as rttm:

            for participant in self.participants:
                for _, val in self.eafFile.tiers[participant][0].items():
                    start = val[0]
                    end = val[1]

                    t0 = self.eafFile.timeslots[start] / sampling_freq
                    length = self.eafFile.timeslots[end] / sampling_freq - t0

                    rttm.write(u"SPEAKER\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format
                               (name, 1, "%.2f" % t0, "%.2f" % length, "<NA>", "<NA>", participant, 1 ))

In [6]:
#!/usr/bin/env python3
import pympi as pmp
import os
import json

TAG_SPANISH = "[- spa]"
TAG_UNDEFINED = "[- und]"
TAG_MIX = "[- mix]"
TAG_CHEQUEAR = "chequear"

class EafFile:
    def _getParticipants(self):
        self.participants = []

        for k in self.eafFile.tiers.keys():
            if 'PARTICIPANT' in self.eafFile.tiers[k][2].keys():
                if self.eafFile.tiers[k][2]['PARTICIPANT'] not in self.participants:
                    self.participants.append(self.eafFile.tiers[k][2]['PARTICIPANT'])

    def _getTiers(self):
        self.tiers = []

        for k in self.eafFile.tiers.keys():
            if 'TIER_ID' in self.eafFile.tiers[k][2].keys():
                if self.eafFile.tiers[k][2]['TIER_ID'] not in self.tiers:
                    self.tiers.append(self.eafFile.tiers[k][2]['TIER_ID'])

    def __init__(self, eafFilePath):
        self.eafPath = eafFilePath
        self.eafFile = pmp.Elan.Eaf(eafFilePath)

        self._getParticipants()
        self._getTiers()

    def toRTTM(self, path_to_write_rttm):
        """
        Function to write a new .rttm file which is a transcription of the .eaf
        given as input
        """
        sampling_freq = 1000

        base = os.path.basename(self.eafPath)
        name = os.path.splitext(base)[0]

        print('Parsing file: {}'.format(name))

        with open(os.path.join(path_to_write_rttm, name + ".rttm"), "w") as rttm:
            for participant in self.participants:
                for _, val in self.eafFile.tiers[participant][0].items():
                    start = val[0]
                    end = val[1]

                    t0 = self.eafFile.timeslots[start] / sampling_freq
                    length = self.eafFile.timeslots[end] / sampling_freq - t0

                    rttm.write(u"SPEAKER\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(
                        name, 1, "%.2f" % t0, "%.2f" % length, "<NA>", "<NA>", participant, 1
                    ))

# Uso del código
eaf_file_path = "ruta/al/archivo.eaf"  # Reemplaza esto con la ruta a tu archivo .eaf
path_to_write_rttm = "ruta/donde/guardar/rttm"  # Reemplaza esto con la ruta donde deseas guardar el archivo .rttm

# Crea una instancia de EafFile y convierte el archivo .eaf a .rttm
eaf_instance = EafFile("casos/ummae-a1-nsb/ummae-a1-nsb.eaf")
eaf_instance.toRTTM("casos/ummae-a1-nsb")


Parsing file: ummae-a1-nsb
